# Install requirements

In [1]:
# Install requirements
!pip install fastapi==0.68.1
!pip install opencv-python==4.5.3.56
# !pip install Pillow==8.3.2
!pip install timm==1.
!pip install python-multipart==0.0.5
!pip install uvicorn==0.15.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 9.7 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Installing build dependencies ... error
error: subprocess-exited-with-error

× pip subprocess to install build dependencies did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.5 MB/s eta

In [2]:
!pip install nest-asyncio

In [3]:
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 12.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=bd38909ae7d108ec6562d6ee1c83b7277f3146ae358f67a036b34615f9ce2316
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok


In [4]:
from typing import List

from fastapi import FastAPI, File, UploadFile
from fastapi.responses import HTMLResponse, StreamingResponse

import cv2
import io
import numpy as np

import torch
import cv2
from PIL import Image

import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure

## DPT Model

In [5]:
def load_model_dpt(model_type):
  ## Load model

  # MiDaS v3 - Large
  # (highest accuracy, slowest inference speed)
  # model_type = "DPT_Large"

  # MiDaS v3 - Hybrid
  # (medium accuracy, medium inference speed)
  # model_type = "DPT_Hybrid"

  # (lowest accuracy, highest inference speed)
  # model_type = "MiDaS_small"  # MiDaS v2.1 - Small

  midas = torch.hub.load("intel-isl/MiDaS", model_type)

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  midas.to(device)
  midas.eval()

  return midas


def pre_process_dpt(image, model_type):

    midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")
    if model_type == "DPT_Large" or model_type == "DPT_Hybrid":
        transform = midas_transforms.dpt_transform
    else:
        transform = midas_transforms.small_transfor

    # Load image
    img = cv2.imdecode(np.frombuffer(image.file.read(),
                                      np.uint8),
                        cv2.IMREAD_COLOR)

    # convert it to the correct format
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Transform it so that it can be used by the model
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    input_batch = transform(img).to(device)

    # Return this image so it can be used in postprocessing
    return input_batch, img

def post_process_dpt(original, prediction):

  prediction = torch.nn.functional.interpolate(
                prediction.unsqueeze(1),
                size=original.shape[:2],
                mode="bicubic",
                align_corners=False,
            ).squeeze()

  output = prediction.cpu().numpy()
  # Create a figure using matplotlib which super-imposes the original
  # image and the prediction

  fig = Figure()
  canvas = FigureCanvas(fig)
  ax = fig.gca()

  # Render both images original as foreground
  ax.imshow(original)
  ax.imshow(output)

  ax.axis("off")
  canvas.draw()

  # Reshape output to be a numpy array
  width, height = fig.get_size_inches() * fig.get_dpi()
  width = int(width)
  height = int(height)
  output_image = np.frombuffer(canvas.tostring_rgb(),
                                dtype='uint8').reshape(height, width, 3)

  # Encode to png
  res, im_png = cv2.imencode(".png", output_image)

  return im_png



# Setup the server

In [6]:
!git clone https://github.com/InterDigitalInc/HRFAE.git
%cd HRFAE

Cloning into 'HRFAE'...
remote: Enumerating objects: 116, done.
remote: Total 116 (delta 0), reused 0 (delta 0), pack-reused 116
Receiving objects: 100% (116/116), 2.79 MiB | 12.91 MiB/s, done.
Resolving deltas: 100% (42/42), done.
/content/HRFAE


In [7]:
%cd logs/001
!./download.sh
%cd ./../..

/content/HRFAE/logs/001
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 49.2M  100 49.2M    0     0   563k      0  0:01:29  0:01:29 --:--:--  594k
/content/HRFAE


In [8]:
import argparse
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import yaml

import matplotlib
matplotlib.use('agg')
from matplotlib import pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

from PIL import Image
from torchvision import transforms, utils

from datasets import *
from nets import *
from functions import *
from trainer import *

In [9]:
parser = argparse.ArgumentParser()
parser.add_argument('--config', type=str, default='001', help='path to the config file.')
parser.add_argument('--vgg_model_path', type=str, default='./models/dex_imdb_wiki.caffemodel.pt', help='pretrained age classifier')
parser.add_argument('--log_path', type=str, default='./logs/', help='log file path')
parser.add_argument('--multigpu', type=bool, default=False, help='use multiple gpus')
parser.add_argument('--checkpoint', type=str, default='', help='checkpoint file path')
parser.add_argument('--img_path', type=str, default='./test/input/', help='test image path')
parser.add_argument('--out_path', type=str, default='./test/output/', help='test output path')
parser.add_argument('--target_age', type=int, default=55, help='Age transform target, interger value between 20 and 70')
opts = parser.parse_known_args()[0]

log_dir = os.path.join(opts.log_path, opts.config) + '/'
if not os.path.exists(opts.out_path):
    os.makedirs(opts.out_path)

config = yaml.safe_load(open('./configs/' + opts.config + '.yaml', 'r'))
img_size = (config['input_w'], config['input_h'])

# Initialize trainer
trainer = Trainer(config)
device = torch.device('cuda')
trainer.to(device)

# Load pretrained model
if opts.checkpoint:
    trainer.load_checkpoint(opts.checkpoint)
else:
    trainer.load_checkpoint(log_dir + 'checkpoint')

def preprocess(img_pil):
    resize = transforms.Compose([
            transforms.Resize(img_size),
            transforms.ToTensor()
            ])
    normalize = transforms.Normalize(mean=[0.48501961, 0.45795686, 0.40760392], std=[1,1,1])

    img_np = np.array(img_pil)
    img = resize(img_pil)
    if img.size(0) == 1:
        img = torch.cat((img, img, img), dim = 0)
    img = normalize(img)
    return img

In [45]:
def is_image(variable):
    try:
        Image.open(variable)
        return True
    except (IOError, OSError):
        return False

In [136]:
import base64

# Code from: https://fastapi.tiangolo.com/tutorial/request-files/
app = FastAPI()


@app.post("/uploadfiles/")
async def create_upload_files(files: List[UploadFile] = File(...)):
    """ Create API endpoint to send image to and specify
     what type of file it'll take

    :param files: Get image files, defaults to File(...)
    :type files: List[UploadFile], optional
    :return: A list of png images
    :rtype: list(bytes)
    """

        # Set target age
    target_age = 65

    with torch.no_grad():
        for file in files:
            contents = await file.read()  # Đọc nội dung của tệp ảnh
            image = Image.open(io.BytesIO(contents))  # Tạo đối tượng Image từ nội dung

            img_name = file.filename
            if not img_name.endswith(('png', 'jpg', 'PNG', 'JPG')):
                print('File ignored: ' + img_name)
                continue
            image_A = preprocess(image)
            image_A = image_A.unsqueeze(0).to(device)

            age_modif = torch.tensor(target_age).unsqueeze(0).to(device)
            image_A_modif = trainer.test_eval(image_A, age_modif, target_age=target_age, hist_trans=True)
            utils.save_image(clip_img(image_A_modif), opts.out_path + img_name.split('.')[0] + '_age_' + str(target_age) + '.jpg')

            # Plot manipulated image
            print(opts.out_path + img_name.split('.')[0] + '_age_' + str(target_age) + '.jpg')

            img_show = Image.open(opts.out_path + img_name.split('.')[0] + '_age_' + str(target_age) + '.jpg')

            img_out = np.array(img_show)
            plt.axis('off')
            plt.imshow(img_out)
            plt.show()

            # Chuyển đổi hình ảnh thành chuỗi byte
            byte_stream = io.BytesIO()
            img_show.save(byte_stream, format='JPEG')
            byte_stream.seek(0)

            # # Trả về hình ảnh dưới dạng streaming response
            # return StreamingResponse(byte_stream, media_type="image/JPEG")

            # Tạo nội dung HTML tùy chỉnh
            content = f"""
                <html>
                <head>
                    <title>Uploaded Image</title>
                    <style>
                        body {{
                            padding: 20px;
                            text-align: center;
                        }}
                        h1 {{
                            color: #333;
                        }}
                        .image-container {{
                            margin-top: 20px;
                        }}
                        .image-container img {{
                            max-width: 100%;
                            height: auto;
                        }}
                    </style>
                </head>
                <body>
                    <h1>Uploaded Image</h1>
                    <div class="image-container">
                        <img src="data:image/jpeg;base64,{base64.b64encode(byte_stream.getvalue()).decode('utf-8')}">
                    </div>
                </body>
                </html>
            """

            # Trả về nội dung HTML tùy chỉnh
            return HTMLResponse(content=content)



In [137]:
@app.get("/")
async def main():
    """Create a basic home page to upload a file

    :return: HTML for homepage
    :rtype: HTMLResponse
    """

    content = """<body>
          <h3>Upload an image to get it's depth map from the MiDaS model</h3>
          <form action="/uploadfiles/" enctype="multipart/form-data" method="post">
              <input name="files" type="file" multiple>
              <input type="submit">
          </form>
      </body>
      """
    return HTMLResponse(content=content)

In [138]:
auth_token = "2SNVvi3DMeCAGfkN1l6lRbqhCg5_5GC4C21zFzmGGvGTDhv2w" #@param {type:"string"}
# Since we can't access Colab notebooks IP directly we'll use
# ngrok to create a public URL for the server via a tunnel

# Authenticate ngrok
# https://dashboard.ngrok.com/signup
# Then go to the "Your Authtoken" tab in the sidebar and copy the API key
import os
os.system(f"ngrok authtoken {auth_token}")

0

In [139]:

from pyngrok import ngrok

# Create tunnel
public_url = ngrok.connect(8000,bind_tls=True)

In [140]:
# Check if it exists
!ps aux | grep ngrok

root       20768  0.0  0.2 736192 27824 ?        Sl   14:48   0:00 /usr/local/lib/python3.10/dist-packages/pyngrok/bin/ngrok start --none --log=stdout
root       20777  0.0  0.0   6904  3216 ?        S    14:48   0:00 /bin/bash -c ps aux | grep ngrok
root       20779  0.0  0.0   6444   660 ?        S    14:48   0:00 grep ngrok


# Make magic happen

In [141]:
import nest_asyncio

# Allow for asyncio to work within the Jupyter notebook cell
nest_asyncio.apply()

import uvicorn

# Run the FastAPI app using uvicorn
print(public_url)
uvicorn.run(app)

NgrokTunnel: "https://e616-34-67-182-18.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [173]
INFO:uvicorn.error:Started server process [173]
INFO:     Waiting for application startup.
INFO:uvicorn.error:Waiting for application startup.
INFO:     Application startup complete.
INFO:uvicorn.error:Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:uvicorn.error:Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     2402:800:63b8:c917:84f2:2bf1:53d2:668c:0 - "GET / HTTP/1.1" 200 OK
INFO:     2402:800:63b8:c917:84f2:2bf1:53d2:668c:0 - "GET /Thay%20%C4%91%E1%BB%95i%20k%C3%ADch%20th%C6%B0%E1%BB%9Bc%20nhi%E1%BB%81u%20%E1%BA%A3nh%20c%C3%B9ng%20l%C3%BAc%21_files/plupload.full.min.js.ta%CC%89i%20xu%C3%B4%CC%81ng HTTP/1.1" 404 Not Found
INFO:     2402:800:63b8:c917:84f2:2bf1:53d2:668c:0 - "GET /Thay%20%C4%91%E1%BB%95i%20k%C3%ADch%20th%C6%B0%E1%BB%9Bc%20nhi%E1%BB%81u%20%E1%BA%A3nh%20c%C3%B9ng%20l%C3%BAc%21_files/app.b7548e8.css HTTP/1.1" 404 Not Found
INFO:     2402:800:63b8:c917:84f2:2bf1:53d2:668c:0 - "GET /Thay%20%C4%91%E1%BB%95i%20k%C3%ADch%20th%C6%B0%E1%BB%9Bc%20nhi%E1%BB%81u%20%E1%BA%A3nh%20c%C3%B9ng%20l%C3%BAc%21_files/esp.js.ta%CC%89i%20xu%C3%B4%CC%81ng HTTP/1.1" 404 Not Found
INFO:     2402:800:63b8:c917:84f2:2bf1:53d2:668c:0 - "GET /Thay%20%C4%91%E1%BB%95i%20k%C3%ADch%20th%C6%B0%E1%BB%9Bc%20nhi%E1%BB%81u%20%E1%BA%A3nh%20c%C3%B9ng%20l%C3%BAc%21_files/f%282%29.txt HTTP/1.1" 404 Not Found


INFO:     Shutting down
INFO:uvicorn.error:Shutting down
INFO:     Waiting for application shutdown.
INFO:uvicorn.error:Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:uvicorn.error:Application shutdown complete.
INFO:     Finished server process [173]
INFO:uvicorn.error:Finished server process [173]


In [142]:
# Kill tunnel
ngrok.disconnect(public_url=public_url)